DataFrame Operations & Transformations in PySpark

By the end of this step, you’ll learn:

How to read and write data in CSV, JSON, and Parquet formats.

How to perform select, filter, groupBy, and joins using the DataFrame API.

How Spark SQL fits in — you can mix SQL and DataFrame APIs.

Best practices: schema inference, repartitioning, and caching.

In [ ]:
"""
Lesson 3: DataFrame Operations and Transformations in PySpark
=============================================================

Author: Deb
Date: 2024-06-15

Description:
------------
This script demonstrates essential PySpark DataFrame operations using the Iris dataset.
It includes downloading the dataset locally, reading it into a Spark DataFrame,
and performing transformations, aggregations, joins, and SQL queries.

Covered Topics:
---------------
1. Creating a SparkSession
2. Downloading data programmatically using urllib
3. Reading CSV data into a DataFrame
4. Performing DataFrame operations:
   - Selecting and filtering columns
   - Adding derived columns
   - Grouping and aggregations
   - Sorting and ordering
   - Joining DataFrames
5. Writing data to Parquet, CSV, and JSON
6. Executing SQL queries on Spark DataFrames

Dataset Used:
-------------
Iris dataset (UCI Machine Learning Repository)
Source: https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv

Usage:
------
1. Activate your virtual environment:
   $ source venv/bin/activate

2. Run the script:
   $ python src/lesson3_dataframe_ops.py

Output:
-------
- Downloads the Iris dataset to /tmp/iris.csv
- Displays sample transformations in console
- Writes transformed DataFrames to output directories (Parquet, CSV, JSON)
"""

# ===============================
# 1. Import required libraries
# ===============================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, expr
import urllib.request
from pathlib import Path


# ===============================
# 2. Create SparkSession
# ===============================
# SparkSession is the unified entry point for Spark operations such as
# reading data, performing transformations, and running SQL queries.

spark = SparkSession.builder \
    .appName("lesson3-dataframe-ops") \
    .master("local[*]") \
    .getOrCreate()

print("✅ Spark Session initialized successfully.")
print("Spark Version:", spark.version)
print("-" * 60)


# ===============================
# 3. Download Iris CSV dataset locally
# ===============================
# This ensures reproducibility and allows the script to run even offline.
# The dataset will be saved to /tmp/iris.csv (works on macOS/Linux).
# On Windows, adjust to Path.home() / "iris.csv" if needed.

iris_url = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
local_path = Path("/tmp/iris.csv")

print("⬇️  Downloading Iris dataset...")
urllib.request.urlretrieve(iris_url, local_path)
print(f"✅ Downloaded dataset to: {local_path}")
print("-" * 60)


# ===============================
# 4. Read CSV file into DataFrame
# ===============================
# inferSchema=True allows Spark to automatically infer column data types.

iris_df = spark.read.csv(str(local_path), header=True, inferSchema=True)

print("=== Schema of Iris Data ===")
iris_df.printSchema()

print("=== First 5 Rows ===")
iris_df.show(5)


# ===============================
# 5. Select & Filter Columns
# ===============================
# Example of basic transformations and filters in PySpark DataFrames.

print("=== Selecting 'sepal_length' and 'species' columns ===")
iris_df.select("sepal_length", "species").show(5)

print("=== Filtering rows where species == 'setosa' ===")
iris_df.filter(iris_df["species"] == "setosa").show(5)

print("=== Filtering rows where sepal_length > 5.0 and species == 'versicolor' ===")
iris_df.filter((iris_df["sepal_length"] > 5.0) & (iris_df["species"] == "versicolor")).show(5)


# ===============================
# 6. Add Derived Column (Feature Engineering)
# ===============================
# Create a new column 'petal_area' = petal_length * petal_width.

iris_df = iris_df.withColumn("petal_area", col("petal_length") * col("petal_width"))

print("=== Added Derived Column: 'petal_area' ===")
iris_df.select("species", "petal_length", "petal_width", "petal_area").show(5)


# ===============================
# 7. GroupBy and Aggregations
# ===============================
# Compute average sepal and petal lengths per species.

agg_df = iris_df.groupBy("species").agg(
    F.avg("sepal_length").alias("avg_sepal_len"),
    F.avg("petal_length").alias("avg_petal_len")
)

print("=== Aggregated Averages by Species ===")
agg_df.show()


# ===============================
# 8. Sorting and Ordering
# ===============================
# Sort by sepal_length in descending order.

print("=== Sorting by sepal_length (descending) ===")
iris_df.orderBy("sepal_length", ascending=False).show(5)


# ===============================
# 9. Joining DataFrames
# ===============================
# Demonstrate joining two DataFrames on a common key (species).

species_data = [
    ("setosa", "short petals"),
    ("versicolor", "medium petals"),
    ("virginica", "long petals"),
]

species_df = spark.createDataFrame(species_data, ["species", "description"])

# Inner join between iris_df and species_df.
joined_df = iris_df.join(species_df, on="species", how="inner")

print("=== Joined DataFrame ===")
joined_df.select("species", "description", "sepal_length", "petal_length").show(5)


# ===============================
# 10. Write Output Data
# ===============================
# Spark writes distributed files, so each output is a directory with multiple part files.

print("=== Writing Data to Disk (CSV, Parquet, JSON) ===")

joined_df.write.mode("overwrite").parquet("output/iris_parquet")
joined_df.write.mode("overwrite").option("header", True).csv("output/iris_csv")
joined_df.write.mode("overwrite").json("output/iris_json")

print("✅ Data successfully written to output/ folder.")
print("-" * 60)


# ===============================
# 11. SQL Interface Example
# ===============================
# Register the DataFrame as a temporary SQL table and run queries directly.

iris_df.createOrReplaceTempView("iris")

sql_result = spark.sql("""
SELECT 
    species,
    AVG(sepal_length) AS avg_sepal_length,
    AVG(petal_length) AS avg_petal_length
FROM iris
GROUP BY species
""")

print("=== SQL Query Results ===")
sql_result.show()


# ===============================
# 12. Stop the Spark Session
# ===============================
# Always stop Spark when done to free resources.

spark.stop()
print("🧠 Spark session stopped. End of Lesson 3.")


✅ Spark Session initialized successfully.
Spark Version: 3.5.7
------------------------------------------------------------
⬇️  Downloading Iris dataset...
✅ Downloaded dataset to: /tmp/iris.csv
------------------------------------------------------------
=== Schema of Iris Data ===
root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)

=== First 5 Rows ===
+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| seto

+-------+------------+------------+------------+
|species| description|sepal_length|petal_length|
+-------+------------+------------+------------+
| setosa|short petals|         5.0|         1.4|
| setosa|short petals|         5.3|         1.5|
| setosa|short petals|         4.6|         1.4|
| setosa|short petals|         5.1|         1.6|
| setosa|short petals|         4.8|         1.4|
+-------+------------+------------+------------+
only showing top 5 rows

=== Writing Data to Disk (CSV, Parquet, JSON) ===
✅ Data successfully written to output/ folder.
------------------------------------------------------------
=== SQL Query Results ===
+----------+-----------------+----------------+
|   species| avg_sepal_length|avg_petal_length|
+----------+-----------------+----------------+
| virginica|6.587999999999998|           5.552|
|versicolor|            5.936|            4.26|
|    setosa|5.005999999999999|           1.464|
+----------+-----------------+----------------+

🧠 Spark sessi